In [17]:
import os
from constants import openai_key
os.environ["OPENAI_API_KEY"]=openai_key

In [18]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import(
    AIMessage,
    SystemMessage,
    HumanMessage
)

In [19]:
speech="""Recurrent neural networks, long short-term memory [12] and gated recurrent [7] neural networks
in particular, have been firmly established as state of the art approaches in sequence modeling and
transduction problems such as language modeling and machine translation [29, 2, 5]. Numerous
efforts have since continued to push the boundaries of recurrent language models and encoder-decoder
architectures [31, 21, 13].
∗Equal contribution. Listing order is random. Jakob proposed replacing RNNs with self-attention and started
the effort to evaluate this idea. Ashish, with Illia, designed and implemented the first Transformer models and
has been crucially involved in every aspect of this work. Noam proposed scaled dot-product attention, multi-head
attention and the parameter-free position representation and became the other person involved in nearly every
detail. Niki designed, implemented, tuned and evaluated countless model variants in our original codebase and
tensor2tensor. Llion also experimented with novel model variants, was responsible for our initial codebase, and
efficient inference and visualizations. Lukasz and Aidan spent countless long days designing various parts of and
implementing tensor2tensor, replacing our earlier codebase, greatly improving results and massively accelerating
our research.
†Work performed while at Google Brain.
‡Work performed while at Google Research.
31st Conference on Neural Information Processing Systems (NIPS 2017), Long Beach, CA, USA.
Recurrent models typically factor computation along the symbol positions of the input and output
sequences. Aligning the positions to steps in computation time, they generate a sequence of hidden
states ht, as a function of the previous hidden state ht−1 and the input for position t. This inherently
sequential nature precludes parallelization within training examples, which becomes critical at longer
sequence lengths, as memory constraints limit batching across examples. Recent work has achieved
significant improvements in computational efficiency through factorization tricks [18] and conditional
computation [26], while also improving model performance in case of the latter. The fundamental
constraint of sequential computation, however, remains.
Attention mechanisms have become an integral part of compelling sequence modeling and transduction models in various tasks, allowing modeling of dependencies without regard to their distance in
the input or output sequences [2, 16]. In all but a few cases [22], however, such attention mechanisms
are used in conjunction with a recurrent network.
In this work we propose the Transformer, a model architecture eschewing recurrence and instead
relying entirely on an attention mechanism to draw global dependencies between input and output.
The Transformer allows for significantly more parallelization and can reach a new state of the art in
translation quality after being trained for as little as twelve hours on eight P100 GPUs.
2 Background
The goal of reducing sequential computation also forms the foundation of the Extended Neural GPU
[20], ByteNet [15] and ConvS2S [8], all of which use convolutional neural networks as basic building
block, computing hidden representations in parallel for all input and output positions. In these models,
the number of operations required to relate signals from two arbitrary input or output positions grows
in the distance between positions, linearly for ConvS2S and logarithmically for ByteNet. This makes
it more difficult to learn dependencies between distant positions [11]. In the Transformer this is
reduced to a constant number of operations, albeit at the cost of reduced effective resolution due
to averaging attention-weighted positions, an effect we counteract with Multi-Head Attention as
described in section 3.2.
Self-attention, sometimes called intra-attention is an attention mechanism relating different positions
of a single sequence in order to compute a representation of the sequence. Self-attention has been
used successfully in a variety of tasks including reading comprehension, abstractive summarization,
textual entailment and learning task-independent sentence representations [4, 22, 23, 19].
End-to-end memory networks are based on a recurrent attention mechanism instead of sequencealigned recurrence and have been shown to perform well on simple-language question answering and
language modeling tasks [28].
To the best of our knowledge, however, the Transformer is the first transduction model relying
entirely on self-attention to compute representations of its input and output without using sequencealigned RNNs or convolution. In the following sections, we will describe the Transformer, motivate
self-attention and discuss its advantages over models such as [14, 15] and [8].
3 Model Architecture
Most competitive neural sequence transduction models have an encoder-decoder structure [5, 2, 29].
Here, the encoder maps an input sequence of symbol representations (x1, ..., xn) to a sequence
of continuous representations z = (z1, ..., zn). Given z, the decoder then generates an output
sequence (y1, ..., ym) of symbols one element at a time. At each step the model is auto-regressive
[9], consuming the previously generated symbols as additional input when generating the next.
The Transformer follows this overall architecture using stacked self-attention and point-wise, fully
connected layers for both the encoder and decoder, shown in the left and right halves of Figure 1,
respectively."""

In [25]:
chat_messages=[
    SystemMessage(content='You are an expert assistant with expertise in summarizing speeches'),
    HumanMessage(content=f'Please provide a short and concise summary of the following speech:\n TEXT:{speech}'),
]


llm=ChatOpenAI(model_name='gpt-3.5-turbo')


In [26]:
llm.get_num_tokens(speech)

1088

In [27]:
llm(chat_messages)

AIMessage(content='The speech discusses the importance of recurrent neural networks, particularly long short-term memory and gated recurrent neural networks, in sequence modeling and transduction problems such as language modeling and machine translation. It introduces the Transformer model as an architecture that relies entirely on an attention mechanism to draw global dependencies between input and output, allowing for more parallelization and achieving state-of-the-art results in translation quality within a short training timeframe. The speech emphasizes the benefits of self-attention in sequence modeling and transduction tasks and describes how the Transformer model differs from traditional models like RNNs and convolutional neural networks.', response_metadata={'token_usage': {'completion_tokens': 117, 'prompt_tokens': 1124, 'total_tokens': 1241}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e3a8f3fb-ef22-4041-9279

In [28]:
print(llm(chat_messages).content)

The speech discusses the advancements in sequence modeling and transduction problems using recurrent neural networks, particularly long short-term memory and gated recurrent neural networks. It introduces the Transformer model architecture, which relies on self-attention mechanisms instead of traditional RNNs or convolution, allowing for more parallelization and improved performance in tasks like language modeling and machine translation. The Transformer model is described as the first transduction model entirely based on self-attention, eliminating the need for sequence-aligned RNNs or convolution. It also explains the encoder-decoder structure and the use of stacked self-attention and fully connected layers in the Transformer model.


PROMPT TEMPLATE TEXT SUMMARISATION

In [29]:
from langchain.chains import LLMChain
from langchain import PromptTemplate

In [30]:
generic_template='''write a summary of the following speech:
speech:`{speech}`
Translate the precise summary to {language}.
'''

prompt=PromptTemplate(
    input_variables=['speech','language'],
    template=generic_template
)

In [31]:
complete_prompt=prompt.format(speech=speech,language='Hindi')

In [32]:
llm.get_num_tokens(complete_prompt)

1106

In [33]:
llm_chain=LLMChain(llm=llm,prompt=prompt)
summary=llm_chain.run({'speech':speech,'language':'Hindi'})

c:\Users\BASAVAPRABHU\anaconda3\envs\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
c:\Users\BASAVAPRABHU\anaconda3\envs\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


In [34]:
summary

'इस भाषण में उत्कृष्ट न्यूरल नेटवर्क, लांग शॉर्ट-टर्म मेमोरी और गेटेड रीकरेंट न्यूरल नेटवर्क्स को स्थापित किया गया है जैसे कि अभिगमन और अनुवाद के समस्याओं में क्रम मॉडेलिंग के लिए राज्य का कला उपाय है। बहुत से प्रयासों ने इसके बाद रिकरेंट भाषा मॉडल्स और एन्कोडर-डीकोडर और्किटेक्चर्स की सीमाएं बढ़ाने का प्रयास किया है। Self-attention के साथ आरएनएन को बदलने की प्रस्तावना की गई और इस विचार का मूल्यांकन किया गया। आटोर्नी मैकेनिजम्स ने अहम भूमिका निभाई है विभिन्न कार्यों में। इस काम में हम ट्रांसफार्मर का प्रस्ताव करते हैं, एक मॉडल आर्किटेक्चर जो पूरी तरह से आरएनएन की आवश्यकता के बजाय ध्यान में एक ध्यान मैकेनिजम का भरोसा करता है। इसके बाद हम मॉडल की विवरण करेंगे, सेल्फ-अटेंशन को प्रेरित करेंगे और इसके लाभों पर चर्चा करेंगे। केंद्रित न्यूरल क्रम अभिक्रमण मॉडल का उपयोग करने का लक्ष्य सीक्वेंस ट्रांसडक्शन मॉडलों में एक एन्कोडर-डीकोडर संरचना का है। इस भाषण में हम विस्तारित न्यूरल जीपीयू, बाइटनेट और कोन्वस 2एस के आधार पर भी चर्चा करेंगे, जिनमें कन्वोलूशनल न्यूरल नेटवर्क का उपयोग किया गया है। इन म

Stuff DocumentChain Text Summarization

In [36]:
from pypdf import PdfReader

In [37]:
# provide the path of  pdf file/files.
pdfreader = PdfReader('PMFBY_Guidelines.pdf')

In [38]:
from typing_extensions import Concatenate
# read text from pdf
text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        text += content

In [39]:
text

' \n \n  \n \nOPERATIONAL GUIDELINES  \n \nPradhan Mantri Fasal Bima Yojana  \n(PMFBY)  \n \n \n \n(Revised)  \n \n \nDepartment of Agriculture, Cooperation and Farmers Welfare  \nMinistry of Agriculture & Farmers Welfare  \nKrishi Bhawan, New Delhi -110001  \n \nIndex  \nSl. No  Section  Page No.  \n                 Abbreviations  I-II \n1 Objective of the Scheme  1 \n2 Adoption of Technology for Scheme Administration  1-2 \n3 Coverage of Farmers  3 \n4 Coverage of Crops  4 \n5 Coverage of Risks & Exclusions  4-5 \n6 Preconditions for implementation of the Scheme  5-6 \n7 Notification  7-11 \n8 Engagement of Common Service Centres and Intermediaries for  \ncoverage of non loanee Farmers  11 \n9 Electronic Remittance of Funds  11-12 \n10 Census code Mapping of Entities  12 \n11 Digitization of Land Records  12 \n12 Sum Insured/Coverage Limit  12 \n13 Premium Rates and Premium Subsidy  13-14 \n14 Budget for Administrative Expenses  14 \n15 Technical Support Unit(TSU)/C entral Programme 

In [40]:
from langchain.docstore.document import Document

In [41]:
docs = [Document(page_content=text)]
docs

[Document(page_content=' \n \n  \n \nOPERATIONAL GUIDELINES  \n \nPradhan Mantri Fasal Bima Yojana  \n(PMFBY)  \n \n \n \n(Revised)  \n \n \nDepartment of Agriculture, Cooperation and Farmers Welfare  \nMinistry of Agriculture & Farmers Welfare  \nKrishi Bhawan, New Delhi -110001  \n \nIndex  \nSl. No  Section  Page No.  \n                 Abbreviations  I-II \n1 Objective of the Scheme  1 \n2 Adoption of Technology for Scheme Administration  1-2 \n3 Coverage of Farmers  3 \n4 Coverage of Crops  4 \n5 Coverage of Risks & Exclusions  4-5 \n6 Preconditions for implementation of the Scheme  5-6 \n7 Notification  7-11 \n8 Engagement of Common Service Centres and Intermediaries for  \ncoverage of non loanee Farmers  11 \n9 Electronic Remittance of Funds  11-12 \n10 Census code Mapping of Entities  12 \n11 Digitization of Land Records  12 \n12 Sum Insured/Coverage Limit  12 \n13 Premium Rates and Premium Subsidy  13-14 \n14 Budget for Administrative Expenses  14 \n15 Technical Support Unit(T

In [42]:
llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [43]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document

In [44]:
template = '''Write a concise and short summary of the following speech.
Speech: `{text}`
'''
prompt = PromptTemplate(
    input_variables=['text'],
    template=template
)

In [46]:
chain = load_summarize_chain(
    llm,
    chain_type='stuff',
    prompt=prompt,
    verbose=False
)


Summarizing Large Documents Using Map Reduce

In [47]:
from langchain import PromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [48]:

# provide the path of  pdf file/files.
pdfreader = PdfReader('PMFBY_Guidelines.pdf')
from typing_extensions import Concatenate
# read text from pdf
text = ''
for i, page in enumerate(pdfreader.pages):
    content = page.extract_text()
    if content:
        text += content

In [49]:

llm = ChatOpenAI(temperature=0, model_name='gpt-3.5-turbo')

In [50]:

## Splittting the text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
chunks = text_splitter.create_documents([text])

In [51]:

len(chunks)

28

In [52]:

chain = load_summarize_chain(
    llm,
    chain_type='map_reduce',
    verbose=False
)
summary = chain.run(chunks)

In [53]:

summary

'The Pradhan Mantri Fasal Bima Yojana (PMFBY) is a crop insurance scheme in India that provides financial support to farmers in case of crop loss or damage. The scheme utilizes technology through the National Crop Insurance Portal for better coordination among stakeholders. Guidelines outline procedures for enrollment, premium payment, claim processing, and dispute resolution, emphasizing transparency and efficiency. The document also details the process of determining loss costs, penalties for insurance companies, evaluation criteria for state departments, and guidelines for sharing information on crop insurance.'

Map reduce with custom prompts

In [54]:
chunks_prompt="""
Please summarize the below speech:
Speech:`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)

In [55]:
final_combine_prompt='''
Provide a final summary of the entire speech with these important points.
Add a Generic Motivational Title,
Start the precise summary with an introduction and provide the
summary in number points for the speech.
Speech: `{text}`
'''
final_combine_prompt_template=PromptTemplate(input_variables=['text'],
                                             template=final_combine_prompt)

In [56]:

summary_chain = load_summarize_chain(
    llm=llm,
    chain_type='map_reduce',
    map_prompt=map_prompt_template,
    combine_prompt=final_combine_prompt_template,
    verbose=False
)
output = summary_chain.run(chunks)

In [57]:
output

'Summary:\n1. The speech emphasizes the operational guidelines for PMFBY, including coverage of farmers, premium rates, digitization of land records, and technology use.\n2. Guidelines for the National Crop Insurance Portal, stakeholder involvement, and monitoring crop health are discussed.\n3. Conditions and guidelines for PMFBY implementation, innovative technology, Aadhaar submission, and bidding process for Insurance Companies are highlighted.\n4. State Governments\' implementation guidelines, historical yield data, and adherence to standards are outlined.\n5. Procedures for enrolling non-loanee farmers, premium rates, subsidies, and payment of claim liabilities are discussed.\n6. Importance of data accuracy, timely communication, and the National Technical Support Unit for PMFBY are emphasized.\n7. Process for data collection, reconciliation, and payment related to crop insurance claims under PMFBY is outlined.\n8. Guidelines for conducting Crop Cutting Experiments, resolving yiel